# Exploratory Data Analysis

Aim: To analyze trends across multiple years of traffic collision datasets and confirm statistical significance.


In [66]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from pprint import pprint

<br>

## Data Acquisition

In [9]:
# Read all dataframes and store in a list
years: list[str] = ["2017", "2018", "2019", "2020", "2021"]

dfs: dict[str, pd.DataFrame] = {year: pd.read_csv(f"../usc_data/sc_unt{year}.csv", low_memory=False) for year in years}

In [16]:
# Length of each dataframe
for year, df in dfs.items():
    print(f"{year}: {len(df):,}")

2017: 268,999
2018: 269,752
2019: 268,227
2020: 224,591
2021: 274,401


In [36]:
# Check which features are available in each dataframe
columns: list[set[str]] = []
for df in dfs.values():
    columns.append(set(df.columns))  # Get all column names

print(f"Number of columns in each dataframe: {[len(columns) for columns in columns]}")

unique_cols: set[str] = columns[0].intersection(*columns)  # Intersection of all column names
print(f"Number of unique columns: {len(unique_cols)}")

Number of columns in each dataframe: [47, 47, 47, 47, 47]
Number of unique columns: 47


<br>

## Data Analysis
Since all columns appear in all dataframes, see the trend of each variable over the years
<br>Aim is to see if there is a trend in the data.

In [44]:
# For each dataframe, add a 'year' column
for year, df in dfs.items():
    df['year'] = year

In [47]:
# Concatenate all dataframes
df: pd.DataFrame = pd.concat(dfs.values(), ignore_index=True)

In [50]:
# Analyze trends for each NUMERIC variable
numeric_cols: pd.Index = df.select_dtypes(include=[np.number]).columns
print(f"Number of numeric columns: {len(numeric_cols)}\n")

trends: dict[str, pd.Series] = {}  # Store mean of each numeric column for each year
for col in numeric_cols:
    trends[col] = df.groupby('year')[col].mean()
    
pd.DataFrame(trends)

Number of numeric columns: 27



,ano,spl,utc,vuc,api,edp,towd,edam,mhe,pd2,...,bdt,soe1,soe2,soe3,soe4,ead,fda,mda,atr2,noc
year,,,,,,,,,,,,,,,,,,,,,
2017,1.751230e+07,42.823821,9.882119,1.610820,7.565233,1702.516843,NaN,2.183384,24.407291,1270.515401,...,13.898116,20.971554,30.205857,38.677496,37.927545,2410.547117,9.860293,11.499745,0.123683,1.361960
2018,1.855833e+07,43.004679,10.028945,1.631806,7.536693,1782.653972,NaN,2.198523,24.529857,1688.594374,...,13.892883,20.975377,30.589912,39.066484,39.055884,2502.059501,9.651604,11.265588,0.122562,1.352665
2019,1.956362e+07,42.842381,10.183147,1.611974,7.471041,1997.934768,NaN,2.197430,24.360437,1648.004316,...,14.032791,21.002889,30.261866,38.812373,38.067112,2534.871199,9.600946,11.400526,0.123629,1.351840
2020,2.036490e+07,42.869119,9.913242,1.629967,7.333313,263.864218,0.475111,2.287126,24.972065,31.330312,...,17.195238,20.766260,31.556665,39.280714,39.240007,2800.612793,9.982207,12.240655,0.513581,1.308454
2021,2.138130e+07,43.258939,10.514346,1.626576,7.320141,200.647662,0.439986,2.291675,24.607636,31.004020,...,20.100394,20.891358,31.066628,39.272846,38.959001,2929.007906,9.995474,12.179970,0.543851,1.312382


In [63]:
# Perform statistical tests
results: dict[str, dict[str, float]] = {}  # Store f-statistic and p-value for each numeric column
for col in numeric_cols:
    f_statistic, p_value = stats.f_oneway(*[group[col].values for name, group in df.groupby('year')])
    results[col] = {'f_statistic': f_statistic, 'p_value': p_value}

pprint(results)

{'ano': {'f_statistic': 20321557.39101855, 'p_value': 0.0},
 'api': {'f_statistic': 53.18099220364458, 'p_value': 6.952150927344713e-45},
 'atg': {'f_statistic': nan, 'p_value': nan},
 'atr2': {'f_statistic': nan, 'p_value': nan},
 'att': {'f_statistic': nan, 'p_value': nan},
 'bdt': {'f_statistic': nan, 'p_value': nan},
 'dtg': {'f_statistic': nan, 'p_value': nan},
 'dtr': {'f_statistic': nan, 'p_value': nan},
 'dtt': {'f_statistic': nan, 'p_value': nan},
 'ead': {'f_statistic': nan, 'p_value': nan},
 'edam': {'f_statistic': 299.81908948392373, 'p_value': 3.007198201730087e-258},
 'edp': {'f_statistic': nan, 'p_value': nan},
 'fda': {'f_statistic': nan, 'p_value': nan},
 'man': {'f_statistic': nan, 'p_value': nan},
 'mda': {'f_statistic': 106.46040724111148, 'p_value': 7.49635469232373e-91},
 'mhe': {'f_statistic': 152.5359172325894, 'p_value': 1.0605886759435779e-130},
 'noc': {'f_statistic': 254.24670618440157, 'p_value': 9.054378168553607e-219},
 'pd2': {'f_statistic': nan, 'p_valu

In [67]:
# Visualize the results

# Create a PdfPages object
pdf_filename = 'stats_results.pdf'
pdf = PdfPages(pdf_filename)

for col, yearly_means in trends.items():
    plt.figure(figsize=(10, 6))
    yearly_means.plot(marker='o')
    plt.title(f'Trend of {col} over years')
    plt.xlabel('Year')
    plt.ylabel('Mean value')
    
    p_value = results[col]['p_value']
    plt.annotate(f'p-value: {p_value:.4f}', xy=(0.05, 0.95), xycoords='axes fraction')
    # plt.show()

    pdf.savefig()  # Save the current figure to the PDF
    plt.close()  # Close the current figure

pdf.close()  # Close the PdfPages object
print(f'All plots are saved to {pdf_filename}')

All plots are saved to stats_results.pdf
